![alt text](pandas.png "Title")

In [1]:
import pandas as pd
import numpy as np
import random

# Dataframes: derive new columns

This is the often core of what we need to do: add new variables in a dataframe. Let's see several ways to do this.

## Test data

In [3]:
patients = [10010, 10011, 10012, 10013]
data = {'gender': ['M', 'F', 'F', 'M'],
        'age':    [20, 40, 20, None],
       }

df = pd.DataFrame(data, index= patients, columns=['age', 'gender'])
df

,age,gender
10010,20.0,M
10011,40.0,F
10012,20.0,F
10013,NaN,M


## Broadcasting

Broadcasting (a NumPy concept) describes how we can combine arrays and scalar values. 

In [4]:
# Create a new df column with the same value in all df rows:
df['study'] = "Study_A123"

# Same, but use element-wise values of another column:
df['age(months)'] = df['age'] * 12

# Using methods on the element-wise values:
df['STUDY'] = df['study'].str.upper() # str accessor brings us method for strings

# Use a regex pattern to extract and broadcast a value:
df['cluster'] = df.study.str.extract('(?:_)(.)') 

# map() applies a function on every iterable item:
df ['a-g'] = df.age.map(str) + '-' + df['gender'] 

df

,age,gender,study,age(months),STUDY,cluster,a-g
10010,20.0,M,Study_A123,240.0,STUDY_A123,A,20.0-M
10011,40.0,F,Study_A123,480.0,STUDY_A123,A,40.0-F
10012,20.0,F,Study_A123,240.0,STUDY_A123,A,20.0-F
10013,NaN,M,Study_A123,NaN,STUDY_A123,A,nan-M


## Using a Python iterable

You can unpack the values from an iterable (e.g. a list, a tuple or a dictionnary). The size of the iterable must be the same of the dataframe.

In [5]:
# Example with a dictionnary:
mydict = {'1': 10, '2': 20, '3': 30, '4': 40}
mydict.keys()

dict_keys(['1', '2', '3', '4'])

In [6]:
df['dict_keys'] = mydict.keys()
df['dict_values'] = mydict.values()
df

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values
10010,20.0,M,Study_A123,240.0,STUDY_A123,A,20.0-M,1,10
10011,40.0,F,Study_A123,480.0,STUDY_A123,A,40.0-F,2,20
10012,20.0,F,Study_A123,240.0,STUDY_A123,A,20.0-F,3,30
10013,NaN,M,Study_A123,NaN,STUDY_A123,A,nan-M,4,40


In [5]:
# Example with lists & tuples
df['newvar']  = [letter for letter in 'abcd']
df['newvar2'] = tuple(letter for letter in 'wxyz')
df

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values,newvar,newvar2
10010,20,M,Study_A123,240,STUDY_A123,A,20-M,1,10,a,w
10011,40,F,Study_A123,480,STUDY_A123,A,40-F,2,20,b,x
10012,20,F,Study_A123,240,STUDY_A123,A,20-F,3,30,c,y
10013,45,M,Study_A123,540,STUDY_A123,A,45-M,4,40,d,z


## Conditional logic

In a SAS datastep there is an implicit 'loop on every record' logic. We can use each variable in an easy way. 

How do we do that in pandas? Let's create a flag in this df: True for males over 40 years, False otherwise.

In [6]:
# Option 1: basic usage of apply() provides an implicit looping on every row.

# This function will get a Series which represents a df row
def create_newvar(row): 
    
    # for readibility
    age = row['age'] 
    gender = row['gender']
    
    # now it feels a bit more like SAS doesn't it?
    if gender == 'M' and age > 40:
        return True
    else:
        return False

# apply() applies a function to each column (if axis = 0) or row (if axis = 1).
# The function returns values which we use to create a new df column.

df['flag'] = df.apply(create_newvar, axis = 1) # or axis = 'columns'
df

# Yes: 1) one new variable at a time, for now.
#      2) axis is confusing. Here's what the doc says:
#         Axis along which the function is applied:
#           0 or ‘index’: apply function to each column.
#           1 or ‘columns’: apply function to each row.
#        Most of the time, we'll use axis=1...

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values,newvar,newvar2,flag
10010,20,M,Study_A123,240,STUDY_A123,A,20-M,1,10,a,w,False
10011,40,F,Study_A123,480,STUDY_A123,A,40-F,2,20,b,x,False
10012,20,F,Study_A123,240,STUDY_A123,A,20-F,3,30,c,y,False
10013,45,M,Study_A123,540,STUDY_A123,A,45-M,4,40,d,z,True


In [7]:
# Alternatively, with a lambda. It's probably the right way to do this small task.
df['flag'] = df.apply(lambda row: True if row['gender']=='M' and row['age']>40 else False, axis = 1)

In [8]:
# Option 2: explicitely iterating on rows, feeling even more like in SAS. Careful, this is not super efficient.

# itterrows() returns a tuple at every iteration: the index value and the row.
for index, row in df.iterrows():
    
    if row['gender'] == 'M' and row['age'] > 40:
        df.loc[index, 'flag'] = True
    
    else:
        df.loc[index, 'flag'] = False
df

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values,newvar,newvar2,flag
10010,20,M,Study_A123,240,STUDY_A123,A,20-M,1,10,a,w,False
10011,40,F,Study_A123,480,STUDY_A123,A,40-F,2,20,b,x,False
10012,20,F,Study_A123,240,STUDY_A123,A,20-F,3,30,c,y,False
10013,45,M,Study_A123,540,STUDY_A123,A,45-M,4,40,d,z,True


In [9]:
# Numpy also provides an easy way: condition/ statement when True, statement when false
df['flag2'] = np.where(df['flag']==True, df['age'] * 12, 'Ignored')
df

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values,newvar,newvar2,flag,flag2
10010,20,M,Study_A123,240,STUDY_A123,A,20-M,1,10,a,w,False,Ignored
10011,40,F,Study_A123,480,STUDY_A123,A,40-F,2,20,b,x,False,Ignored
10012,20,F,Study_A123,240,STUDY_A123,A,20-F,3,30,c,y,False,Ignored
10013,45,M,Study_A123,540,STUDY_A123,A,45-M,4,40,d,z,True,540


## Create multiple variables in one go

In [10]:
# This is probably what we want...

def create_newvars(row): 
    
    if row['gender'] == 'M' and row['age'] > 40:
        
        # Return a dictionary: keys = names of the future df variables.
        # We could have passed a list instead but then you can't choose the var names and must rename afterwards
        return {'Flag1': True, 'Flag2': 'cat1'}
    else:
        return {'Flag1': False, 'Flag2': 'cat2'}

newvars = df.apply(create_newvars, axis='columns', result_type='expand')

# newvars df contains only the new variables, let's add these cols to the original df
df = pd.concat([df, newvars], axis='columns')
df

,age,gender,study,age(months),STUDY,cluster,a-g,dict_keys,dict_values,newvar,newvar2,flag,flag2,Flag1,Flag2
10010,20,M,Study_A123,240,STUDY_A123,A,20-M,1,10,a,w,False,Ignored,False,cat2
10011,40,F,Study_A123,480,STUDY_A123,A,40-F,2,20,b,x,False,Ignored,False,cat2
10012,20,F,Study_A123,240,STUDY_A123,A,20-F,3,30,c,y,False,Ignored,False,cat2
10013,45,M,Study_A123,540,STUDY_A123,A,45-M,4,40,d,z,True,540,True,cat1


In [11]:
# split() can also be useful to split a column into others
df['test'] = "firstname-lastname"
df[['Firstname','Lastname']] = df.test.str.split('-', expand=True)
df[['test', 'Firstname','Lastname']]

,test,Firstname,Lastname
10010,firstname-lastname,firstname,lastname
10011,firstname-lastname,firstname,lastname
10012,firstname-lastname,firstname,lastname
10013,firstname-lastname,firstname,lastname


## Discretization: create categorical variables

In [12]:
# cut() uses a list of bins to create an interval variable:
bins = [10, 20, 30, 40, 100]
df['agegr'] = pd.cut(df['age'], bins, right=True)
df[['age','agegr']]

# ( or ) are exclusive, [ or ] are inclusive. Change the 'right' value to revert this

,age,agegr
10010,20,"(10, 20]"
10011,40,"(30, 40]"
10012,20,"(10, 20]"
10013,45,"(40, 100]"


In [13]:
# We could pass our own group names:
bins = [10, 20, 30, 40, 100]
groups = ['teens', 'young adults', 'adults', 'aging adults']
df['agegr'] = pd.cut(df['age'], bins, labels=groups)
df[['age','agegr']]

,age,agegr
10010,20,teens
10011,40,adults
10012,20,teens
10013,45,aging adults


In [14]:
# Somehow related, we can simply compare values:
df['flag'] = df['age'].gt(30) # True if age is greater than 30, False otherwise. 
df[['age','flag']]

# Also available: eq(), ge(), lt() and le()

,age,flag
10010,20,False
10011,40,True
10012,20,False
10013,45,True


## Change from baseline

In [8]:
# let's create a VS dataframe
def create_vs():
    patients = [10010, 10011, 10013]
    visits = [1, 2, 3]
    param = ['heart rate', 'systolic blood pressure']

    data = {'subjid': sorted(patients * len(visits)) * len(param),
            'visit' : visits * len(param) * len(patients),
            'param' : sorted(param * len(visits) * len(patients)),
            'result': [random.randint(50, 150)  for n in range(len(visits) * len(patients))] +
                      [random.randint(100, 180) for n in range(len(visits) * len(patients))] 
    }

    return pd.DataFrame(data, columns=['subjid', 'visit', 'param', 'result']).sort_values(['subjid','param', 'visit']).reset_index()
    
vs=create_vs()
vs.head()

,index,subjid,visit,param,result
0,0,10010,1,heart rate,67
1,1,10010,2,heart rate,120
2,2,10010,3,heart rate,103
3,9,10010,1,systolic blood pressure,101
4,10,10010,2,systolic blood pressure,148


In [16]:
# Let's add baseline flag at visit 1:

# I'm using a lambda function because it's a small & unique task...
vs['bslfl'] = vs.apply(lambda row: True if row['visit']==1 else False, axis = 1)
vs.head()

,subjid,visit,param,result,bslfl
0,10010,1,heart rate,108,True
1,10010,2,heart rate,108,False
2,10010,3,heart rate,93,False
9,10010,1,systolic blood pressure,139,True
10,10010,2,systolic blood pressure,144,False


In [17]:
# Change from previous visit.

# shift() is the equivalent of the lag function in SAS. Combines well with a groupby().
vs['shift'] = vs.groupby(['subjid', 'param'])['result'].shift(periods=1) # periods = number of rows to shift
vs['chgbsl'] = vs['result'] - vs['shift']
vs.head(8)

,subjid,visit,param,result,bslfl,shift,chgbsl
0,10010,1,heart rate,108,True,NaN,NaN
1,10010,2,heart rate,108,False,108.0,0.0
2,10010,3,heart rate,93,False,108.0,-15.0
9,10010,1,systolic blood pressure,139,True,NaN,NaN
10,10010,2,systolic blood pressure,144,False,139.0,5.0
11,10010,3,systolic blood pressure,122,False,144.0,-22.0
3,10011,1,heart rate,56,True,NaN,NaN
4,10011,2,heart rate,62,False,56.0,6.0


In [18]:
# In fact, diff() does the difference between the original and the shifted values.
vs['diff'] = vs.groupby(['subjid', 'param'])['result'].diff() # diff(periods=1) by default
vs.head(5)

# As high level as it gets :-)

,subjid,visit,param,result,bslfl,shift,chgbsl,diff
0,10010,1,heart rate,108,True,NaN,NaN,NaN
1,10010,2,heart rate,108,False,108.0,0.0,0.0
2,10010,3,heart rate,93,False,108.0,-15.0,-15.0
9,10010,1,systolic blood pressure,139,True,NaN,NaN,NaN
10,10010,2,systolic blood pressure,144,False,139.0,5.0,5.0


In [87]:
# Let's now create a better Baseline flag and a Change from Baseline

# Let's add a few missings. We want the baseline flag to be True for the earliest visit with a result, by subjid & param.
vs=create_vs()

vs.loc[0, 'result']= None
vs.loc[4, 'result']= None
vs.loc[9, 'result']= None
vs.head(10)

,index,subjid,visit,param,result
0,0,10010,1,heart rate,NaN
1,1,10010,2,heart rate,124.0
2,2,10010,3,heart rate,81.0
3,9,10010,1,systolic blood pressure,158.0
4,10,10010,2,systolic blood pressure,NaN
5,11,10010,3,systolic blood pressure,177.0
6,3,10011,1,heart rate,95.0
7,4,10011,2,heart rate,106.0
8,5,10011,3,heart rate,103.0
9,12,10011,1,systolic blood pressure,NaN


In [88]:
# Create the baseline flag
baselines = pd.DataFrame(vs[vs['result'].notnull()].groupby(['subjid', 'param'])['visit'].min()).reset_index()
baselines['baseline'] = True
baselines

,subjid,param,visit,baseline
0,10010,heart rate,2,True
1,10010,systolic blood pressure,1,True
2,10011,heart rate,1,True
3,10011,systolic blood pressure,2,True
4,10013,heart rate,1,True
5,10013,systolic blood pressure,1,True


In [89]:
# Merge flag to vs
vs = vs.merge(baselines, how='left', on=['subjid', 'param', 'visit'])
vs['baseline'].fillna(False, inplace=True)
vs

,index,subjid,visit,param,result,baseline
0,0,10010,1,heart rate,NaN,False
1,1,10010,2,heart rate,124.0,True
2,2,10010,3,heart rate,81.0,False
3,9,10010,1,systolic blood pressure,158.0,True
4,10,10010,2,systolic blood pressure,NaN,False
5,11,10010,3,systolic blood pressure,177.0,False
6,3,10011,1,heart rate,95.0,True
7,4,10011,2,heart rate,106.0,False
8,5,10011,3,heart rate,103.0,False
9,12,10011,1,systolic blood pressure,NaN,False


In [90]:
# What was the result value at baseline?
baselines = vs[ vs['baseline']].rename(columns={'result': 'bsl_value'}) [['subjid', 'param', 'bsl_value']]
baselines

,subjid,param,bsl_value
1,10010,heart rate,124.0
3,10010,systolic blood pressure,158.0
6,10011,heart rate,95.0
10,10011,systolic blood pressure,168.0
12,10013,heart rate,99.0
15,10013,systolic blood pressure,105.0


In [91]:
vs = vs.merge(baselines, how='left', on=['subjid', 'param'])

In [92]:
# Finally calculate the change from baseline
vs['Chg_bsl'] = vs['result'] - vs['bsl_value']
vs

,index,subjid,visit,param,result,baseline,bsl_value,Chg_bsl
0,0,10010,1,heart rate,NaN,False,124.0,NaN
1,1,10010,2,heart rate,124.0,True,124.0,0.0
2,2,10010,3,heart rate,81.0,False,124.0,-43.0
3,9,10010,1,systolic blood pressure,158.0,True,158.0,0.0
4,10,10010,2,systolic blood pressure,NaN,False,158.0,NaN
5,11,10010,3,systolic blood pressure,177.0,False,158.0,19.0
6,3,10011,1,heart rate,95.0,True,95.0,0.0
7,4,10011,2,heart rate,106.0,False,95.0,11.0
8,5,10011,3,heart rate,103.0,False,95.0,8.0
9,12,10011,1,systolic blood pressure,NaN,False,168.0,NaN


## Ranks

Ranking assign ranks (numbers) from 1 to the number of data points. In case of ties, by default rank is assigned to the mean rank (but there are more options)

In [12]:
data = {
    'subjid': ['S01', 'S02', 'S03', 'S04'],
    'age':    [20, 30, 30, 40],
}

df = pd.DataFrame(data, columns=['subjid', 'age'])
df

,subjid,age
0,S01,20
1,S02,30
2,S03,30
3,S04,40


In [15]:
# series Age : 20 years old has rank 1, 40 has rank 4 and 30 has mean rank from 2 to 3. This is not in place
df['age'].rank()


0    1.0
1    2.5
2    2.5
3    4.0
Name: age, dtype: float64

In [16]:
# in a dataframe: this is not in place
df_rank = df.rank()
df_rank

,subjid,age,rank
0,1.0,1.0,1.0
1,2.0,2.5,2.5
2,3.0,2.5,2.5
3,4.0,4.0,4.0


__________________________________________________
Nicolas Dupuis, Methodology and Innovation (IDAR C&SP), 2020+